In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
df = pd.read_csv(r"E:\Projects\liver_cirrhosis_stage\liver_cirrhosis_stage\liver_cirrhosis.csv")

In [3]:
df.head()

,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,2221,C,Placebo,18499,F,N,Y,N,N,0.5,149.0,4.04,227.0,598.0,52.70,57.0,256.0,9.9,1
1,1230,C,Placebo,19724,M,Y,N,Y,N,0.5,219.0,3.93,22.0,663.0,45.00,75.0,220.0,10.8,2
2,4184,C,Placebo,11839,F,N,N,N,N,0.5,320.0,3.54,51.0,1243.0,122.45,80.0,225.0,10.0,2
3,2090,D,Placebo,16467,F,N,N,N,N,0.7,255.0,3.74,23.0,1024.0,77.50,58.0,151.0,10.2,2
4,2105,D,Placebo,21699,F,N,Y,N,N,1.9,486.0,3.54,74.0,1052.0,108.50,109.0,151.0,11.5,1


In [4]:
df.describe

<bound method NDFrame.describe of        N_Days Status             Drug    Age Sex Ascites Hepatomegaly Spiders  \
0        2221      C          Placebo  18499   F       N            Y       N   
1        1230      C          Placebo  19724   M       Y            N       Y   
2        4184      C          Placebo  11839   F       N            N       N   
3        2090      D          Placebo  16467   F       N            N       N   
4        2105      D          Placebo  21699   F       N            Y       N   
...       ...    ...              ...    ...  ..     ...          ...     ...   
24995    3584      D  D-penicillamine  23612   F       N            N       N   
24996    3584      D  D-penicillamine  23612   F       N            N       N   
24997     971      D  D-penicillamine  16736   F       N            Y       Y   
24998    3707      C  D-penicillamine  16990   F       N            Y       N   
24999    3707      C  D-penicillamine  16990   F       N            Y      

In [5]:
df.info

<bound method DataFrame.info of        N_Days Status             Drug    Age Sex Ascites Hepatomegaly Spiders  \
0        2221      C          Placebo  18499   F       N            Y       N   
1        1230      C          Placebo  19724   M       Y            N       Y   
2        4184      C          Placebo  11839   F       N            N       N   
3        2090      D          Placebo  16467   F       N            N       N   
4        2105      D          Placebo  21699   F       N            Y       N   
...       ...    ...              ...    ...  ..     ...          ...     ...   
24995    3584      D  D-penicillamine  23612   F       N            N       N   
24996    3584      D  D-penicillamine  23612   F       N            N       N   
24997     971      D  D-penicillamine  16736   F       N            Y       Y   
24998    3707      C  D-penicillamine  16990   F       N            Y       N   
24999    3707      C  D-penicillamine  16990   F       N            Y       N

Now, we will proceed with data preprocessing

In [6]:
#Null removal
df.dropna(inplace=True)

In [7]:
df.shape

(25000, 19)

In [8]:
#Outlier removal
numeric_cols = df.select_dtypes(include=['number']).columns.tolist()

Q1 = df[numeric_cols].quantile(0.25)
Q3 = df[numeric_cols].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outlier_indices = df[((df[numeric_cols] < lower_bound) | (df[numeric_cols] > upper_bound)).any(axis=1)].index

df.drop(outlier_indices, inplace=True)

In [9]:
df.shape

(15361, 19)

In [10]:
#Normalization
from sklearn.preprocessing import MinMaxScaler
numeric_cols_for_normalization = df.select_dtypes(include=['number']).columns.tolist()
if 'Stage' in numeric_cols_for_normalization:
    numeric_cols_for_normalization.remove('Stage')
scaler = MinMaxScaler()
df[numeric_cols_for_normalization] = scaler.fit_transform(df[numeric_cols_for_normalization])


In [11]:
df.head()

,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
1,0.250105,C,Placebo,0.531493,M,Y,N,Y,N,0.029412,0.193732,0.741379,0.097826,0.122744,0.099237,0.229630,0.353774,0.486486,2
2,0.871477,C,Placebo,0.117625,F,N,N,N,N,0.029412,0.481481,0.517241,0.255435,0.313095,0.554022,0.266667,0.365566,0.270270,2
3,0.431005,D,Placebo,0.360540,F,N,N,N,N,0.058824,0.296296,0.632184,0.103261,0.241221,0.290076,0.103704,0.191038,0.324324,2
4,0.434161,D,Placebo,0.635156,F,N,Y,N,N,0.235294,0.954416,0.517241,0.380435,0.250410,0.472108,0.481481,0.191038,0.675676,1
8,0.518090,C,Placebo,0.297449,F,N,N,N,N,0.294118,0.622537,0.741379,0.108696,0.505087,0.617733,0.597794,0.952830,0.513514,1


Now , lets use few algorithms and then later choose the best one in the end

In [12]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [13]:
X = df.drop('Stage', axis=1)
y = df['Stage']

In [14]:
categorical_cols = X.select_dtypes(include='object').columns

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_features = encoder.fit_transform(X[categorical_cols])
encoded_feature_names = encoder.get_feature_names_out(categorical_cols)
encoded_df = pd.DataFrame(encoded_features, columns=encoded_feature_names, index=X.index)

X = X.drop(categorical_cols, axis=1)
X = pd.concat([X, encoded_df], axis=1)

print(f"Shape of features (X) after one-hot encoding: {X.shape}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training data shape: {X_train.shape}, Testing data shape: {X_test.shape}")

model_accuracies = {}

Shape of features (X) after one-hot encoding: (15361, 27)
Training data shape: (12288, 27), Testing data shape: (3073, 27)


In [15]:
#Logistic Regression
param_grid_lr = {'C': [0.1, 1.0, 10.0]}
grid_search_lr = GridSearchCV(LogisticRegression(solver='liblinear', max_iter=200), param_grid_lr, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_lr.fit(X_train, y_train)
best_lr_model = grid_search_lr.best_estimator_
y_pred_lr = best_lr_model.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
model_accuracies['Logistic Regression'] = accuracy_lr
print(f"Best Logistic Regression Parameters: {grid_search_lr.best_params_}")
print(f"Logistic Regression Accuracy: {accuracy_lr:.4f}")

Best Logistic Regression Parameters: {'C': 1.0}
Logistic Regression Accuracy: 0.6007


In [17]:
#Decision Tree
param_grid_dt = {'max_depth': [5, 10, 15], 'min_samples_leaf': [1, 5, 10]}
grid_search_dt = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid_dt, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_dt.fit(X_train, y_train)
best_dt_model = grid_search_dt.best_estimator_
y_pred_dt = best_dt_model.predict(X_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
model_accuracies['Decision Tree'] = accuracy_dt
print(f"Best Decision Tree Parameters: {grid_search_dt.best_params_}")
print(f"Decision Tree Accuracy: {accuracy_dt:.4f}")

Best Decision Tree Parameters: {'max_depth': 15, 'min_samples_leaf': 1}
Decision Tree Accuracy: 0.9079


In [18]:
#Random Forest
param_grid_rf = {'n_estimators': [50, 100], 'max_depth': [10, 20]}
grid_search_rf = GridSearchCV(RandomForestClassifier(random_state=42, n_jobs=-1), param_grid_rf, cv=5, scoring='accuracy')
grid_search_rf.fit(X_train, y_train)
best_rf_model = grid_search_rf.best_estimator_
y_pred_rf = best_rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
model_accuracies['Random Forest'] = accuracy_rf
print(f"Best Random Forest Parameters: {grid_search_rf.best_params_}")
print(f"Random Forest Accuracy: {accuracy_rf:.4f}")

Best Random Forest Parameters: {'max_depth': 20, 'n_estimators': 100}
Random Forest Accuracy: 0.9476


In [19]:
#Gradient Boosting
param_grid_gb = {'n_estimators': [50, 100], 'learning_rate': [0.05, 0.1]}
grid_search_gb = GridSearchCV(GradientBoostingClassifier(random_state=42), param_grid_gb, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_gb.fit(X_train, y_train)
best_gb_model = grid_search_gb.best_estimator_
y_pred_gb = best_gb_model.predict(X_test)
accuracy_gb = accuracy_score(y_test, y_pred_gb)
model_accuracies['Gradient Boosting'] = accuracy_gb
print(f"Best Gradient Boosting Parameters: {grid_search_gb.best_params_}")
print(f"Gradient Boosting Accuracy: {accuracy_gb:.4f}")

Best Gradient Boosting Parameters: {'learning_rate': 0.1, 'n_estimators': 100}
Gradient Boosting Accuracy: 0.8604


In [20]:
#SVM
param_grid_svc = {'C': [0.1, 1.0], 'kernel': ['linear', 'rbf']} 
grid_search_svc = GridSearchCV(SVC(random_state=42), param_grid_svc, cv=3, scoring='accuracy', n_jobs=-1) 
grid_search_svc.fit(X_train, y_train)
best_svc_model = grid_search_svc.best_estimator_
y_pred_svc = best_svc_model.predict(X_test)
accuracy_svc = accuracy_score(y_test, y_pred_svc)
model_accuracies['SVC'] = accuracy_svc
print(f"Best SVC Parameters: {grid_search_svc.best_params_}")
print(f"SVC Accuracy: {accuracy_svc:.4f}")

Best SVC Parameters: {'C': 1.0, 'kernel': 'rbf'}
SVC Accuracy: 0.7319


In [24]:
print("\nModel Performance Summary")
sorted_models = sorted(model_accuracies.items(), key=lambda item: item[1], reverse=True)

for model_name, accuracy in sorted_models:
    print(f"{model_name}: {accuracy:.4f}")

print("\nTop 5 Models by Accuracy")
for model_name, accuracy in sorted_models:
    print(f"{model_name} (Accuracy: {accuracy:.4f})")


Model Performance Summary
Random Forest: 0.9476
Decision Tree: 0.9079
Gradient Boosting: 0.8604
SVC: 0.7319
Logistic Regression: 0.6007

Top 5 Models by Accuracy
Random Forest (Accuracy: 0.9476)
Decision Tree (Accuracy: 0.9079)
Gradient Boosting (Accuracy: 0.8604)
SVC (Accuracy: 0.7319)
Logistic Regression (Accuracy: 0.6007)
